# 🐍 CSV to JSON Konverter für Matrix_v1

In [1]:
import pandas as pd
import json
import os
import pprint

In [2]:
script_dir = os.getcwd()
csv_file = os.path.join(script_dir, "Matrix_v1.csv")

# CSV einlesen mit "," als Trennzeichen und doppelten Anführungszeichen
df = pd.read_csv(csv_file, sep=",", quotechar='"', dtype=str).fillna("")
df.head()


,ID,Bezeichnung,Beschreibung,URL,NetzwerkeServices,DigitaleAnwendungen,Empfehlungen,RechercheRessourcen,KulturSammlungsgutKolonialeKontexte,NsverfolgungsbedingtEntzogenesKulturgut,KulturgutentziehungenSBZDDR,KriegsbedingtVerlagertesKulturgut,HumanRemains,SecretSacredObjects,NaturkundeNaturwissenschaft
0,JREAD3,Proveana,Proveana ist die Datenbank des Deutschen Zentr...,https://www.proveana.de/de/start,,x,,x,x,x,x,x,x,x,x
1,RC5RQ2,CCC-Portal - Sammlungsgut aus kolonialen Konte...,Die endgültige Version des von der DDB betrieb...,https://ccc.deutsche-digitale-bibliothek.de/de,,x,,x,x,,,,,x,x
2,SVIYS2,Archivführer zur deutschen Kolonialzeit,Deutschlands koloniale Vergangenheit hat vielf...,https://archivfuehrer-kolonialzeit.de/,,x,,x,x,,,,,,
3,BXCKBE,PAESE - Provenienzforschung in außereuropäisch...,Die PAESE-Verbunddatenbank versammelte von 202...,https://www.postcolonial-provenance-research.com/,x,x,,x,x,,,,,,
4,JS950W,Deutsche Digitale Bibliothek (DDB),"Gemälde, Bücher, Musikstücke, Skulpturen, Film...",https://www.deutsche-digitale-bibliothek.de,,x,,x,x,x,x,x,x,x,


In [3]:
# Definition: Textfelder
text_fields = ["ID", "Bezeichnung", "Beschreibung", "URL"]

# Mapping der Spalten zu OCMDP-IDs
column_to_ocmdp = {
    "NsverfolgungsbedingtEntzogenesKulturgut": "A01ISV",
    "KulturSammlungsgutKolonialeKontexte": "IP5W9R",
    "KulturgutentziehungenSBZDDR": "D1ZG69",
    "KriegsbedingtVerlagertesKulturgut": "IVETAX",
    "SecretSacredObjects": "Z5V1LB",
    "HumanRemains": "SZMZE4",
    "NaturkundeNaturwissenschaft": "A0VF7A",
    "NetzwerkeServices": "WXJBI5",
    "DigitaleAnwendungen": "PSEBRV",
    "Empfehlungen": "UOVWQQ",
    "RechercheRessourcen": "UQBLZP"
}

# OCMDP-URI-Präfix
prefix = "http://www.w3id.org/objectcore/terminology/"

# Gruppierung: Was ist Kategorie, was ist Typ
category_keys = {
    "KulturSammlungsgutKolonialeKontexte",
    "NsverfolgungsbedingtEntzogenesKulturgut",
    "KulturgutentziehungenSBZDDR",
    "KriegsbedingtVerlagertesKulturgut",
    "HumanRemains",
    "SecretSacredObjects",
    "NaturkundeNaturwissenschaft"
}

type_keys = {
    "NetzwerkeServices",
    "DigitaleAnwendungen",
    "Empfehlungen",
    "RechercheRessourcen"
}

# Alle Booleschen Felder: alle außer den Textfeldern
bool_fields = [col for col in df.columns if col not in text_fields]


In [4]:
# === JSON 1: Mit Boolean-Feldern ===
records = []
for i, (_, row) in enumerate(df.iterrows(), start=1):
    obj = {
        "internalID": i,
        **{field: row[field] for field in text_fields},
        **{field: row[field].strip().lower() == "x" for field in bool_fields}
    }
    records.append(obj)

In [5]:
# === JSON 2: Mit category/type URIs ===
extended_records = []
for i, row in enumerate(df.itertuples(index=False), start=1):
    base = {
        "internalID": i,
        **{field: getattr(row, field) for field in text_fields}
    }

    categories = []
    types = []

    for col, ocmdp_id in column_to_ocmdp.items():
        val = getattr(row, col, "").strip().lower()
        if val == "x":
            uri = prefix + ocmdp_id
            if col in category_keys:
                categories.append(uri)
            elif col in type_keys:
                types.append(uri)

    if categories:
        base["category"] = categories
    if types:
        base["type"] = types

    extended_records.append(base)

In [6]:
# === Dateien schreiben ===
print("📄 Inhalt von Matrix_v1.json (Booleans):")
pprint.pprint(records[:3], sort_dicts=False)  # Zeige die ersten 3 zum Überblick

print("\n📄 Inhalt von Matrix_v1_extended.json (category/type URIs):")
pprint.pprint(extended_records[:3], sort_dicts=False)  # Zeige die ersten 3 zum Überblick


📄 Inhalt von Matrix_v1.json (Booleans):
[{'internalID': 1,
  'ID': 'JREAD3',
  'Bezeichnung': 'Proveana',
  'Beschreibung': 'Proveana ist die Datenbank des Deutschen Zentrums '
                  'Kulturgutverluste für die Provenienzforschung. Sie stellt '
                  'insbesondere Ergebnisse der vom Zentrum geförderten '
                  'Forschungsprojekte dar. Über die Website kann nach '
                  'Personen, Körperschaften, Ereignissen, Sammlungen, '
                  'Provenienzmerkmalen, Objekten und weiterführenden Quellen '
                  'recherchiert werden. Sie enthält außerdem Schwerpunkte mit '
                  'Informationen zu Themen wie „Jüdische Sammler:innen“, '
                  '„Kunstfund Gurlitt“ oder „Verlustkataloge kriegsbedingt '
                  'verlagerter Kulturgüter“. Proveana durchsucht auch die '
                  'Inhalte der Lost Art-Datenbank und stellt außerdem '
                  'Verknüpfungen zu anderen Datenbanken her. Provean